添加包 
抑制报错

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

np.set_printoptions(suppress=True)
import warnings 
warnings.filterwarnings('ignore')


In [ ]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

数据分析

In [ ]:
train.head()

In [ ]:
train.info()
train.Stay.unique()

In [ ]:
train.isnull().sum().sort_values(ascending=False)


In [ ]:
test.isnull().sum().sort_values(ascending=False)

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
for i in train.columns:
    print(i, ':', train[i].nunique())
    
# nunique 返回不同值的个数
# unique 返回不同值的数组

In [ ]:
# Number of distinct observations in test dataset
for i in test.columns:
    print(i, ':', test[i].nunique())

data preparation 

In [20]:
train['Bed Grade'].fillna(train['Bed Grade'].mode()[0],inplace=True)
test['Bed Grade'].fillna(test['Bed Grade'].mode()[0],inplace=True)

train['City_Code_Patient'].fillna(train['City_Code_Patient'].mode()[0], inplace = True)
test['City_Code_Patient'].fillna(test['City_Code_Patient'].mode()[0], inplace = True)
# mode 众数 
# param(true) 默认不考虑缺省值

label encoding Stay column in train dataset 

In [21]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['Stay'] = le.fit_transform(train['Stay'].astype('str'))

In [22]:
train.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,4
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,3
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,4
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,4


In [28]:
test.columns


Index(['case_id', 'Hospital_code', 'Hospital_type_code', 'City_Code_Hospital',
       'Hospital_region_code', 'Available Extra Rooms in Hospital',
       'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed Grade',
       'patientid', 'City_Code_Patient', 'Type of Admission',
       'Severity of Illness', 'Visitors with Patient', 'Age',
       'Admission_Deposit'],
      dtype='object')

In [31]:
#Imputing dummy Stay column in test datset to concatenate with train dataset
test['Stay'] = -1 
df = pd.concat([train, test])
df.shape

# 这里的test以全部都是-1填充

(455495, 18)

In [32]:
#Label Encoding all the columns in Train and test datasets
for i in ['Hospital_type_code', 'Hospital_region_code', 'Department',
          'Ward_Type', 'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness', 'Age']:
    le = LabelEncoder()
    df[i] = le.fit_transform(df[i].astype(str))

In [33]:
#Spearating Train and Test Datasets
train = df[df['Stay']!=-1]
test = df[df['Stay']==-1]

feature engineering 

In [34]:
def get_countid_enocde(train, test, cols, name):
  temp = train.groupby(cols)['case_id'].count().reset_index().rename(columns = {'case_id': name})
  train = pd.merge(train, temp, how='left', on= cols)
  train[name] = train[name].astype('float')
  train[name].fillna(np.median(temp[name]), inplace = True)
  
  temp2 = test.groupby(cols)['case_id'].count().reset_index().rename(columns = {'case_id': name})
  test = pd.merge(test,temp2, how='left', on= cols)
  test[name] = test[name].astype('float')
  test[name].fillna(np.median(temp2[name]), inplace = True)
  return train, test

### 这个函数的实现需要再确认

In [35]:
train, test = get_countid_enocde(train, test, ['patientid'], name = 'count_id_patient')
train, test = get_countid_enocde(train, test, 
                                 ['patientid', 'Hospital_region_code'], name = 'count_id_patient_hospitalCode')
train, test = get_countid_enocde(train, test, 
                                 ['patientid', 'Ward_Facility_Code'], name = 'count_id_patient_wardfacilityCode')

In [36]:
# Droping duplicate columns
test1 = test.drop(['Stay', 'patientid', 'Hospital_region_code', 'Ward_Facility_Code'], axis =1)
train1 = train.drop(['case_id', 'patientid', 'Hospital_region_code', 'Ward_Facility_Code'], axis =1)

In [37]:
# Splitting train data for Naive Bayes and XGBoost
X1 = train1.drop('Stay', axis =1)
y1 = train1['Stay']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size =0.20, random_state =100)

构建模型

In [38]:
from sklearn.naive_bayes import GaussianNB
target = y_train.values
features = X_train.values
classifier_nb = GaussianNB()
model_nb = classifier_nb.fit(features, target)


In [39]:
prediction_nb = model_nb.predict(X_test)
from sklearn.metrics import accuracy_score
acc_score_nb = accuracy_score(prediction_nb,y_test)
print("Acurracy:", acc_score_nb*100)

Acurracy: 34.55439015199096


In [40]:
import xgboost
classifier_xgb = xgboost.XGBClassifier(max_depth=4, learning_rate=0.1, n_estimators=800,
                                  objective='multi:softmax', reg_alpha=0.5, reg_lambda=1.5,
                                  booster='gbtree', n_jobs=4, min_child_weight=2, base_score= 0.75)

In [41]:
model_xgb = classifier_xgb.fit(X_train, y_train)

In [42]:
prediction_xgb = model_xgb.predict(X_test)
acc_score_xgb = accuracy_score(prediction_xgb,y_test)
print("Accuracy:", acc_score_xgb*100)

Accuracy: 43.047355859816605


In [52]:
# Segregation of features and target variable
X = train.drop('Stay', axis =1)
y = train['Stay']
print(X.columns)
z = test.drop('Stay', axis = 1)
print(z.columns)

# Data Scaling
from sklearn import preprocessing
X_scale = preprocessing.scale(X)
X_scale.shape

Index(['case_id', 'Hospital_code', 'Hospital_type_code', 'City_Code_Hospital',
       'Hospital_region_code', 'Available Extra Rooms in Hospital',
       'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed Grade',
       'patientid', 'City_Code_Patient', 'Type of Admission',
       'Severity of Illness', 'Visitors with Patient', 'Age',
       'Admission_Deposit', 'count_id_patient',
       'count_id_patient_hospitalCode', 'count_id_patient_wardfacilityCode'],
      dtype='object')
Index(['case_id', 'Hospital_code', 'Hospital_type_code', 'City_Code_Hospital',
       'Hospital_region_code', 'Available Extra Rooms in Hospital',
       'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed Grade',
       'patientid', 'City_Code_Patient', 'Type of Admission',
       'Severity of Illness', 'Visitors with Patient', 'Age',
       'Admission_Deposit', 'count_id_patient',
       'count_id_patient_hospitalCode', 'count_id_patient_wardfacilityCode'],
      dtype='object')


(318438, 20)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size =0.20, random_state =100)

In [58]:
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [59]:
from keras.utils import to_categorical
#Sparse Matrix
a = to_categorical(y_train)
b = to_categorical(y_test)

In [60]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape = (254750, 20))) 
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(11, activation='softmax'))

In [61]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 254750, 64)        1344      
                                                                 
 dense_13 (Dense)            (None, 254750, 128)       8320      
                                                                 
 dense_14 (Dense)            (None, 254750, 256)       33024     
                                                                 
 dense_15 (Dense)            (None, 254750, 512)       131584    
                                                                 
 dense_16 (Dense)            (None, 254750, 512)       262656    
                                                                 
 dense_17 (Dense)            (None, 254750, 11)        5643      
                                                                 
Total params: 442571 (1.69 MB)
Trainable params: 44257

In [62]:
model.compile(optimizer= 'SGD', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [63]:
callbacks = [tf.keras.callbacks.TensorBoard("logs_keras")]
model.fit(X_train, a, epochs=20, callbacks=callbacks, validation_split = 0.2)

Epoch 1/20


ValueError: in user code:

    File "d:\anaconda\envs\conda_gpu\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "d:\anaconda\envs\conda_gpu\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        
    File "d:\anaconda\envs\conda_gpu\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        
    File "d:\anaconda\envs\conda_gpu\Lib\site-packages\keras\src\engine\training.py", line 1150, in train_step
        
    File "d:\anaconda\envs\conda_gpu\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        actionable for you (your own code).
    File "d:\anaconda\envs\conda_gpu\Lib\site-packages\keras\src\engine\input_spec.py", line 298, in assert_input_compatibility
        

    ValueError: Input 0 of layer "sequential_2" is incompatible with the layer: expected shape=(None, 254750, 20), found shape=(None, 20)


In [69]:
model = Sequential()
model.add(tf.keras.Input(shape=(20,)))
model.add(Dense(64, activation='relu')) 
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(11, activation='softmax'))

model.compile(optimizer= 'SGD', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [70]:
callbacks = [tf.keras.callbacks.TensorBoard("logs_keras")]
model.fit(X_train, a, epochs=20, callbacks=callbacks, validation_split = 0.2)

Epoch 1/20


6358/6369 [============================>.] - ETA: 0s - loss: 1.6619 - accuracy: 0.3670WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000002F64F5D3880> and will run it as-is.
Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x000002F64F5D3880>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x000002F64F5D3880>. Note that functions defined in certain environments, like the i

In [73]:
# Retraining the model with 4 epochs
model.fit(X_train, a, epochs=4, validation_split = 0.2)
print("\n Model Evaluation")
model.evaluate(X_test,b)

Epoch 1/4
6369/6369 [==============================] - 16s 2ms/step - loss: 1.4699 - accuracy: 0.4332 - val_loss: 1.5068 - val_accuracy: 0.4199
Epoch 2/4
6369/6369 [==============================] - 16s 2ms/step - loss: 1.4677 - accuracy: 0.4336 - val_loss: 1.5053 - val_accuracy: 0.4229
Epoch 3/4
6369/6369 [==============================] - 15s 2ms/step - loss: 1.4651 - accuracy: 0.4348 - val_loss: 1.5064 - val_accuracy: 0.4218
Epoch 4/4
6369/6369 [==============================] - 15s 2ms/step - loss: 1.4634 - accuracy: 0.4360 - val_loss: 1.5090 - val_accuracy: 0.4201

 Model Evaluation
1991/1991 [==============================] - 3s 1ms/step - loss: 1.5042 - accuracy: 0.4200


AttributeError: 'Sequential' object has no attribute 'predict_classes'